### Step 1: Setup local configuration

Configure workspace and the resource group

In [ ]:
!az configure --defaults workspace=<Your workspace> group=<Your resource group>

### Step 2: Create AML environment to capture WOFS python dependencies

#### Generate environment configuration yaml

In [ ]:
%%writefile wofs_docker_env.yaml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: wofs_docker_env/
build:
  path: wofs_docker_env

#### Create the environment using CLI

In [ ]:
!az ml environment create --file wofs_docker_env.yaml

### Step 3: Start the training job

In [ ]:
%%writefile wofs_training_job.yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: ./src
command: >-
   ./train_wofs.sh ${{inputs.zarr}} ${{inputs.norm_stats}} ${{outputs.output_dir}}; 
inputs:
  zarr: 
    type: uri_folder
    path: azureml:azureml_unzip_sampled_output_data_output_dir:1
  norm_stats:
    type: uri_folder
    path: azureml:wofs_norm_stats:1
outputs:
  output_dir:
    type: uri_folder
    mode: rw_mount
services:
    my_vs_code:
      type: vs_code
      nodes: all 
    my_jupyter_lab:
      type: jupyter_lab
      nodes: all
environment: azureml:wofs_docker_env@latest
compute: azureml:Standard-NC48ads-A100-v4
experiment_name: wofs
display_name: wofs_train_model
description: wofs_train_model


In [ ]:
!az ml job create -f wofs_training_job.yaml